# Using data selection and transformation functions
The `covid19pandas` package has built-in functions that allow you to easily perform several types of data selections and transformations. This tutorial summarizes how to use each of them individually.

The real power of the functions is seen when you compose them together and then pass the results to one of our built-in plotting functions (or one of your own). For examples of this, see the quickstart tutorial.

First, import the package.

In [1]:
import covid19pandas as cod

This is the table we're going to use for this demo:

In [2]:
global_df = cod.get_data_jhu()

# Select only the columns we need for now. We don't need the
# additional location ID columns like Latitutde, Longitude, and UID.
global_df = global_df[["date", "Country/Region", "Province/State", "Combined_Key", "cases", "deaths", "recovered"]]
global_df.tail()

These data were obtained from Johns Hopkins University (https://github.com/CSSEGISandData/COVID-19).


,date,Country/Region,Province/State,Combined_Key,cases,deaths,recovered
26131,2020-04-29,West Bank and Gaza,NaN,WestBankandGaza,344,2,71
26132,2020-04-29,Western Sahara,NaN,WesternSahara,6,0,5
26133,2020-04-29,Yemen,NaN,Yemen,6,0,1
26134,2020-04-29,Zambia,NaN,Zambia,97,3,54
26135,2020-04-29,Zimbabwe,NaN,Zimbabwe,32,4,5


# Data selection functions
The two built-in data selection functions are `select_regions` and `select_top_x_regions`.

## `select_regions`
This function allows you to get data for specific regions out of a table. These are the main parameters:
- `data`: The dataframe to select from.
- `region_col`: The name of the column to filter rows by.
- `regions`: A list of values. Select rows that have one of these values in the column named `region_col`.

For example, to get all rows for Italy or Spain from the table we're working with, you would pass `"Country/Region"` to `region_col` and `["Italy", "Spain"]` to `regions`:

In [3]:
italy_spain = cod.select_regions(
    data=global_df,
    region_col="Country/Region",
    regions=["Italy", "Spain"])

italy_spain.tail()

,date,Country/Region,Province/State,Combined_Key,cases,deaths,recovered
25569,2020-04-27,Spain,NaN,Spain,229422,23521,120832
25757,2020-04-28,Italy,NaN,Italy,201505,27359,68941
25833,2020-04-28,Spain,NaN,Spain,232128,23822,123903
26021,2020-04-29,Italy,NaN,Italy,203591,27682,71252
26097,2020-04-29,Spain,NaN,Spain,236899,24275,132929


What if one of the columns we're selecting data for has multiple subregions, but we just want one overall total for the whole country for each day? `select_regions` can perform this aggregation automatically. Simply pass `True` to the optional `combine_subregions` parameter. You will also need to provide a list of which data columns you want to have aggregated. Pass this list to the optional `data_cols` parameter. Any other data columns will be dropped.

To select the data for China and the United States and combine the counts into one total for each country for each day:

In [4]:
china_usa = cod.select_regions(
    data=global_df,
    region_col="Country/Region",
    regions=["China", "US"],
    combine_subregions=True,
    data_cols=["cases", "deaths", "recovered"])

china_usa.tail()

,date,Country/Region,cases,deaths,recovered
193,2020-04-27,US,988197,56259,111424
194,2020-04-28,China,83940,4637,78422
195,2020-04-28,US,1012582,58355,115936
196,2020-04-29,China,83944,4637,78474
197,2020-04-29,US,1039909,60967,120720


## `select_top_x_regions`
This function allows you to select the regions regions that have the highest current counts in a particular data type. These are the parameters:
- `data`: The dataframe to select from.
- `data_type`: The data type (e.g. cases, deaths, or recovered) by which to rank countries.
- `region_cols`: Columns to use to group rows into different regions. You can pass a single column, or a list of columns.
- `x`: The number of top regions to select, e.g. pass `10` to get the top 10.

To get the data for the top 10 countries with the most cumulative cases as of the most recent day:

In [5]:
top_ten = cod.select_top_x_regions(
    data=global_df,
    data_type="cases",
    region_cols="Country/Region",
    x=10)

top_ten.tail(10)

,date,Country/Region,cases
980,2020-04-29,China,83944
981,2020-04-29,France,166543
982,2020-04-29,Germany,161539
983,2020-04-29,Iran,93657
984,2020-04-29,Italy,203591
985,2020-04-29,Russia,99399
986,2020-04-29,Spain,236899
987,2020-04-29,Turkey,117589
988,2020-04-29,US,1039909
989,2020-04-29,United Kingdom,166441


### Exclude particular countries
If you want to exclude particular regions from being selected in the table, simply pass them to the optional `exclude` parameter. Any regions that have a value in any of the `region_cols` that matches a value in the `exclude` list will be dropped, and the next highest regions will fill their place.

In [6]:
top_ten_excl = cod.select_top_x_regions(
    data=global_df,
    data_type="cases",
    region_cols="Country/Region",
    x=10,
    exclude=["US", "China"])

top_ten_excl.tail(10)

,date,Country/Region,cases
980,2020-04-29,Belgium,47859
981,2020-04-29,Brazil,79685
982,2020-04-29,France,166543
983,2020-04-29,Germany,161539
984,2020-04-29,Iran,93657
985,2020-04-29,Italy,203591
986,2020-04-29,Russia,99399
987,2020-04-29,Spain,236899
988,2020-04-29,Turkey,117589
989,2020-04-29,United Kingdom,166441


### Control automatic aggregation
You may have noticed that `select_top_x_regions` automatically aggregated the counts for regions with multiple subregions, and in the process dropped the data types that we weren't ranking countries by. There are optional parameters we can use to control these behaviors.

To keep the other data types, pass them to the `other_data_cols` parameter as a list. They will also be aggregated.

In [7]:
top_ten_all_data = cod.select_top_x_regions(
    data=global_df,
    data_type="cases",
    region_cols="Country/Region",
    x=10,
    other_data_cols=["deaths", "recovered"])

top_ten_all_data.tail(10)

,date,Country/Region,cases,deaths,recovered
980,2020-04-29,China,83944,4637,78474
981,2020-04-29,France,166543,24121,49118
982,2020-04-29,Germany,161539,6467,120400
983,2020-04-29,Iran,93657,5957,73791
984,2020-04-29,Italy,203591,27682,71252
985,2020-04-29,Russia,99399,972,10286
986,2020-04-29,Spain,236899,24275,132929
987,2020-04-29,Turkey,117589,3081,44040
988,2020-04-29,US,1039909,60967,120720
989,2020-04-29,United Kingdom,166441,26166,857


And to prevent any aggregation at all, pass `False` to the `combine_subregions` parameter:

In [8]:
top_ten_uncombined = cod.select_top_x_regions(
    data=global_df,
    data_type="cases",
    region_cols="Country/Region",
    x=10,
    combine_subregions=False)

top_ten_uncombined.tail()

,date,Country/Region,Province/State,Combined_Key,cases,deaths,recovered
26122,2020-04-29,United Kingdom,Gibraltar,"Gibraltar,UnitedKingdom",141,0,131
26123,2020-04-29,United Kingdom,Isle of Man,"IsleofMan,UnitedKingdom",313,21,258
26124,2020-04-29,United Kingdom,Montserrat,"Montserrat,UnitedKingdom",11,1,2
26125,2020-04-29,United Kingdom,Turks and Caicos Islands,"TurksandCaicosIslands,UnitedKingdom",12,1,5
26126,2020-04-29,United Kingdom,NaN,UnitedKingdom,165221,26097,0


# Data transformation functions
The three built-in data transformation functions are `calc_x_day_rolling_mean`, `calc_daily_change`, and `calc_days_since_min_count`. "Transformation" is perhaps a bit of a misnomer, because these functions keep the original data values, and add the new values as a new column.

## `calc_x_day_rolling_mean`
Sometimes there can be a lot of jitter in data from day to day. A rolling mean transformation takes the value for each day and averages it with the values immediately surrounding it. This helps smooth things out.

These are the main parameters for the `calc_x_day_rolling_mean` function:
- `data`: The dataframe to calculate from.
- `data_types`: The data types (e.g. cases, deaths, or recovered) to calculate means for. Pass a single data type or a list.
- `region_cols`: The column or columns that are required to uniquely identify each row for each day. This function won't do any aggregation for you.
- `x`: The number of days to use for each mean. For example, `x=3` will average the value for each day with the values for the day before and the day afterwards.

To calculate the 3 day rolling mean for all the data types in our example table:

In [9]:
means_3_days = cod.calc_x_day_rolling_mean(
    data=global_df,
    data_types=["cases", "deaths", "recovered"],
    region_cols=["Combined_Key"],
    x=3)

means_3_days.tail()

,date,Country/Region,Province/State,Combined_Key,cases,deaths,recovered,mean_cases,mean_deaths,mean_recovered
26131,2020-04-29,West Bank and Gaza,NaN,WestBankandGaza,344,2,71,343.000000,2.0,75.0
26132,2020-04-29,Western Sahara,NaN,WesternSahara,6,0,5,6.000000,0.0,5.0
26133,2020-04-29,Yemen,NaN,Yemen,6,0,1,2.666667,0.0,1.0
26134,2020-04-29,Zambia,NaN,Zambia,97,3,54,93.333333,3.0,46.0
26135,2020-04-29,Zimbabwe,NaN,Zimbabwe,32,4,5,32.000000,4.0,5.0


By default, each value will be averaged with the `x` - 1 values before it. However, you may want each value to be averaged with values on either side of it. This is referred to as a "centered window". To get this instead, pass `True` to the optional `center` parameter.

## `calc_daily_change`
The tables in this package by default come with the cumulative counts for each day. However, you may be interested in just looking at the rate of change in the counts, or in other words just look at the new cases/deaths/recovered for each day. This is similar to the idea of the first derivative for a continuous function. It's useful because, for example, counts of cumulative cases include everyone who ever tested positive for the disease, even if they've already recovered or passed away.

The `calc_daily_change` function has three parameters:
- `data`: The dataframe to calculate from.
- `data_types`: The columns to calculate the daily change for. Pass a single column or a list. Other data columns will be left unchanged.
- `region_cols`: The column or columns that are required to uniquely identify each row for each day. This function won't do any aggregation for you.

To generate a column with the daily change in cases for each day:

In [10]:
daily_cases = cod.calc_daily_change(
    data=global_df,
    data_types="cases",
    region_cols="Combined_Key")

daily_cases.tail()

,date,Country/Region,Province/State,Combined_Key,cases,deaths,recovered,daily_cases
26131,2020-04-29,West Bank and Gaza,NaN,WestBankandGaza,344,2,71,1
26132,2020-04-29,Western Sahara,NaN,WesternSahara,6,0,5,0
26133,2020-04-29,Yemen,NaN,Yemen,6,0,1,5
26134,2020-04-29,Zambia,NaN,Zambia,97,3,54,2
26135,2020-04-29,Zimbabwe,NaN,Zimbabwe,32,4,5,0


## `calc_days_since_min_count`
Since different regions started having infections at different times, sometimes it's useful to compare them based on the number of days since they reached a minimum number of cases, deaths, or recovered. That way, you can see how different compare to each other at similar points. This function helps you do that. (Remember to account for population differences--see the quickstart tutorial for an example of that.)

The `calc_days_since_min_count` has the following parameters:
- `data`: The dataframe to calculate from.
- `data_type`: The column for the data type to look for the minmium count in.
- `region_cols`: The column or columns that are required to uniquely identify each row for each day. This function won't do any aggregation for you.
- `min_count`: The minimum count we're going start counting at.

This function will drop rows for days where a country had less than the minimum count.

To generate a column with the number of days since each country had 100 cases:

In [11]:
days_since_100 = cod.calc_days_since_min_count(
    data=global_df,
    data_type="cases",
    region_cols="Combined_Key",
    min_count=100)

days_since_100.tail()

,date,Country/Region,Province/State,Combined_Key,cases,deaths,recovered,days_since_100_cases
8242,2020-04-29,Vietnam,NaN,Vietnam,270,0,222,38
8311,2020-04-29,West Bank and Gaza,NaN,WestBankandGaza,344,2,71,31
8255,2020-04-29,Australia,Western Australia,"WesternAustralia,Australia",551,8,507,38
8307,2020-04-29,China,Yunnan,"Yunnan,China",185,2,181,87
8308,2020-04-29,China,Zhejiang,"Zhejiang,China",1268,1,1263,94


## Further reading
To get the full documentation on any function, simply pass it to the Python `help` function:

In [12]:
help(cod.calc_x_day_rolling_mean)

Help on function calc_x_day_rolling_mean in module covid19pandas.selectors:

calc_x_day_rolling_mean(data, data_types, region_cols, x, center=False)
    Calculate a centered rolling mean with x days for each number in a count.
    
    Parameters:
    data (pandas.DataFrame): The data to calculate the rolling means for.
    data_types (str or list of str): The data columns in your table that you want to calculate the x day rolling means for.
    region_cols (str or list of str): Column(s) that uniquely identify each region for each day.
    x (int): The number of days to calculate the means over.
    center (bool, optional): Whether to center the window on each value, instead of having the value at the right side of the window. Default False.
    
    Returns:
    pandas.DataFrame: The table, with rolling means calculated over the specified number of days.

